In [ ]:
#爬取的电脑网页链接https://weibo.com/2610874582/HdRqGwZNv
#对应的手机网页链接https://m.weibo.cn/2610874582/4333036507864049（手机网页和客户端地址一样）
#博主说微博在手机网页端比较容易获取（这个应该是经验之谈，我记住就好）

import requests
import json
import os
url='https://m.weibo.cn/comments/hotflow?id=4333036507864049&mid=4333036507864049&max_id_type=0'

r=requests.get(url)
#print(r.text)

#获取下一页网页ID地址信息
data_biaoqing=json.loads(r.text)
next_url_id=data_biaoqing.get('data').get('max_id')

#获取表情包图片地址
root='picturepractice/'#定义一个根目录
for i in range(18):
    url_face=data_biaoqing.get('data').get('data')[i].get('pic').get('large').get('url')
   # print(url_face)
    path=root+url_face.split('/')[-1]#url.split('/')[-1]指URL反斜杠分隔的最后一段
    try:
        if not os.path.exists(root):#判断当前根目录存不存在，不存在就建立一个
            os.mkdir(root)
        if not os.path.exists(path):#判断这个文件是否粗存在
           r=requests.get(url_face)
           with open(path,'wb') as f:
                f.write(r.content)
                f.close()
                print("文件保存成功")
        else:
         print("文件已存在")
    except:
         print("文件爬取失败")
            
#小结-芜湖，成功啦，成功爬取首页评论的18条评论中的图片（表情包），存储在根文件夹
#picturepractice/文件夹下。下面的代码尝试抓取前十页评论的所有图片

In [3]:
#为上面代码的进阶练习，爬取前十页评论的所有表情包
import requests
import json
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time

url='https://m.weibo.cn/comments/hotflow?id=4333036507864049&mid=4333036507864049&max_id_type=0'
#login_url='https://passport.weibo.cn/sso/login'
#login_data={'username':'18850555804',#表单数据
#            'password':'linhuiqin199304'}


#session=requests.Session()#定义session类
#r=session.post(login_url,login_data)#这步只是为了获取表单数据
#r_cookie=session.cookies#获取登陆后的表单数据
#print('11111111111Cookies',r_cookie)
#r=session.get(url)
#print(r.text)



#获取下一页网页ID地址信息
def get_everypage_url():
    my_loginname='18850555804'
    my_password='linhuiqin199304'
    #用selenium
    browser=webdriver.Chrome()
    browser.get('https://passport.weibo.cn/signin/welcome?entry=mweibo&r=https%3A%2F%2Fm.weibo.cn%2Fcomments%2Fhotflow%3Fid%3D4333036507864049%26max_id%3D174979505889554')
    #browser.find_element_by_class_name('btn btnWhite').click()#通过class名字定位节点并点击
    browser.find_element(By.XPATH,'//div[@class="action"]/a[2]').click()
    #跳转网页到登陆页面
    time.sleep(2)
    browser.find_element(By.XPATH,'//p[@class="input-box"]/input').send_keys(my_loginname)
    browser.find_element(By.ID,'loginPassword').send_keys(my_password)#传入账号密码
    browser.find_element(By.ID,'loginAction').click()#点击登录
    #输入账号密码并点击了登陆，跳转到获取短信验证码页面
    time.sleep(2)
    browser.find_element(By.XPATH,'//div[@class="my-btn-box"]/a').click()#这里是点击获取短信验证码
    #接下来要传入短信验证码
    time.sleep(10)
    yanzheng=input("输入六位验证码：")#把六位验证码赋值到yanzheng中
    time.sleep(2)
    browser.find_element(By.XPATH,'//span[@class="my-input fz16 fc-33 my-td-txt"]/input').send_keys(yanzheng)#成功把验证码输入到表格中
    time.sleep(15)
    browser.find_element(By.XPATH,'//div[@class="my-btn-box"]/a').click() #10.14我感觉是可以了。再验证下看能不能行的通
    

    url_sum=[]
    url1='https://m.weibo.cn/comments/hotflow?id=4333036507864049&mid=4333036507864049&max_id_type=0'
    url_sum.append(url1)
#    print(type(url_sum),url_sum)
#    print(url_sum[0],type(url_sum))
    #初始网页单独定义
    for i in range(5): 
        r_everypage=browser.get(str(url_sum[i]))   
        data_biaoqing=json.loads(r_everypage.text)
        next_url_id=data_biaoqing.get('data').get('max_id')
        url_next='https://m.weibo.cn/comments/hotflow?id=4333036507864049&max_id='+str(next_url_id)#+'&max_id_type=0'
        print(url_next,type(url_next))
        url_sum.append(url_next)
        print(url_sum,'第',i+1,'次循环')
#获取一页表情包图片地址，传入该页获取内容，
def onepage_faceget(data_biaoqing):
    root='picturepractice/'#定义一个根目录
    for i in range(18):
        url_face=data_biaoqing.get('data').get('data')[i].get('pic').get('large').get('url')
       # print(url_face)这个是获取data_biaoqing内的图片地址
        path=root+url_face.split('/')[-1]#url.split('/')[-1]指URL反斜杠分隔的最后一段
        try:
            if not os.path.exists(root):#判断当前根目录存不存在，不存在就建立一个
                os.mkdir(root)
            if not os.path.exists(path):#判断这个文件是否粗存在
               r=requests.get(url_face)
               with open(path,'wb') as f:
                    f.write(r.content)
                    f.close()
                    print("文件保存成功")
            else:
                print("文件已存在")
        except:
            print("文件爬取失败")
            

if __name__=='__main__':
    get_everypage_url()
  

The chromedriver version (117.0.5938.62) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.70); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


AttributeError: 'NoneType' object has no attribute 'text'

In [2]:
#input函数使用方法，由于input是python的内建函数，因此无需额外导入
yanzheng=input("输入六位验证码：")
print('已接受键入数据',yanzheng,type(yanzheng))



已接受键入数据 999999 <class 'str'>


In [ ]:
#微博账号密码登陆的方式太麻烦了，一天只能登陆3次，太局限
#https://m.weibo.cn/login?backURL=https%3A%2F%2Fm.weibo.cn%2F
#通过短信验证码方式登陆，需要解决的问题是他会跳出验证码框，需要....有点难，目前没找到教程